# Requirements

In this exercise we will make use of the ``memory_profiler`` package.

In [ ]:
%pip install memory_profiler

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Exercise 1

Implement the random walk class below which allows a user to iterate over the steps in a 1-dimensional random walk, i.e. the step position $x_n$ at timestep $n$ is given by:

\begin{align}
x_n = x_{n - 1} + \Delta x,
\end{align}

where $\Delta x$ is a sample from a normal distribution $\mathcal{N}(0, 1)$ with mean $0$ and standard deviation $1$.


## a)

In [ ]:
import random
from collections.abc import Iterable, Iterator

class RandomWalkIterator(Iterator):
    """
    Iterator for steps in a random walk.
    """
    def __init__(self, walk):
        self.walk = walk
        self.index = 0

    def __next__(self):
        if self.index < self.walk.n_steps:
            item = self.walk.steps[self.index]
            self.index += 1
            return item
        raise StopIteration()

    def __iter__(self):
        return self

class RandomWalkIterable(Iterable):
    """
    A 1-dimensional random walk with unit step size.
    """
    def __init__(self, n_steps = 1000):
        """
        Args:
            n_steps: The number of random walk steps to perform.
        """
        pos = 0.0
        self.n_steps = n_steps
        self.steps = []
        for i in range(n_steps):
            pos += random.gauss(0, 1)
            self.steps.append(pos)

    def __iter__(self):
        return RandomWalkIterator(self)

Then, use the code below to profile the memory consumption of the calculating the standard deviation of the random walk. Discuss the form of the memory profile.

In [ ]:
def calculate_std(random_walk):
    """
    Calculate standard deviation of a random walk.

    Args:
        random_walk(``Iterable``): An iterable over the steps of
             a random walk.
    """
    n = 0
    step_squared_sum = 0.0
    step_sum = 0.0
    for step in random_walk:
        step_sum += step
        step_squared_sum += step * step
        n += 1
    return (step_squared_sum - step_sum) / n


In [ ]:
def calculate_random_walk_iterator():
    return calculate_std(RandomWalkIterable(10_000_000))

from memory_profiler import memory_usage
memory_iterator = memory_usage(calculate_random_walk_iterator)

In [ ]:
time = 0.1 * np.arange(len(memory_iterator))
plt.plot(time, memory_iterator)
plt.xlabel("Time [s]")
plt.ylabel("Memory used [MB]")

## b)

Complete the code below to implement a generator version of the random walk code. Remember that you can use ``yield`` to simplify your code.

In [ ]:
class RandomWalkGenerator:
    def __init__(self, n_steps = 1000):
        self.pos = 0.0
        self.step = 0
        self.n_steps = n_steps

    def __iter__(self):
        for i in range(self.n_steps - 1):
            current_pos = self.pos
            self.pos += random.gauss(0, 1)
            yield current_pos
        return self.pos

Then, use the code below to compar the memory profiles of the two implementations. How do they differ?

In [ ]:
def calculate_random_walk_generator():
    return calculate_std(RandomWalkGenerator(10_000_000))

In [ ]:
memory_generator = memory_usage(calculate_random_walk_generator)

In [ ]:
time_iterator = 0.1 * np.arange(len(memory_iterator))
time_generator = 0.1 * np.arange(len(memory_generator))
plt.plot(time_iterator, memory_iterator, label="Iterator")
plt.plot(time_generator, memory_generator, label="Generator")
plt.xlabel("Time [s]")
plt.ylabel("Memory used [MB]")
plt.legend()
plt.savefig("figures/memory_used.pdf")

# Exercise 2

Complete the two functions to filter positive steps from the random walk. For the first one use a =for= loop and
for the second a list comprehension.

In [ ]:
def filter_positive_loop(random_walk):
    positive_steps = []
    for step in random_walk:
        positive_steps.append(step)
    return positive_steps

In [ ]:
def filter_positive_comprehension(random_walk):
    return [r for r in random_walk if r > 0.0]


In [ ]:
random_walk = RandomWalkIterable(1_000_000)

In [ ]:
%timeit filter_positive_loop(random_walk)

In [ ]:
%timeit filter_positive_loop(random_walk.steps)

In [ ]:
%timeit filter_positive_comprehension(random_walk)

In [ ]:
%timeit filter_positive_comprehension(random_walk.steps)

# Exercise 3

Write a decorator ``@maximum_memory`` that prints out the maximum amount of memory required by during execution of a function.

**Note:** You can forward arugments to the ``memory_usage`` function by passing a tuple ``(f, args, kwargs)``
    containing the function ``f`` to call, the list of positional arguments ``args`` and the dictionary
    of keyword args ``kwargs``.

In [ ]:
def maximum_memory(f):
    def wrapper(*args, **kwargs):
        memory = memory_usage((f, args, kwargs))
        print(f"Maximum memory: {max(memory)}")
    return wrapper

In [ ]:
@maximum_memory
def calculate_random_walk_decorated(n_steps):
    calculate_std(RandomWalkIterable(n_steps))

In [ ]:
calculate_random_walk_decorated(10000000)

# Exercise 4

Apply the flyweight pattern to reduce the memory footprint of the ``RandomWalkIterableClass``.

In [ ]:
import random
from collections.abc import Iterable, Iterator

class RandomWalkIterator(Iterator):
    """
    Iterator for steps in a random walk.
    """
    def __init__(self, walk):
        self.walk = walk
        self.index = 0

    def __next__(self):
        if self.index < self.walk.n_steps:
            item = self.walk.steps[self.index]
            self.index += 1
            return item
        raise StopIteration()

    def __iter__(self):
        return self

class RandomWalkIterable(Iterable):
    """
    A 1-dimensional random walk with unit step size.
    """
    _steps = []
    _n_steps = 0
    def __new__(cls, n_steps):
        if (n_steps > cls._n_steps):
            cls._steps = cls._calculate_random_walk(n_steps)
            cls._n_steps = n_steps
        random_walk = super().__new__(cls)
        random_walk.n_steps = n_steps
        random_walk.steps = cls._steps
        return random_walk
            
    @staticmethod
    def _calculate_random_walk(n_steps):
        pos = 0.0
        steps = []
        for i in range(n_steps):
            steps.append(pos)
            pos += random.gauss(0, 1)
        return steps
        
    
    def __init__(self, n_steps = 1000):
        pass
            
        
    def __iter__(self):
        for i in range(self.n_steps):
            yield self.steps[i]

In [ ]:
def calculate_random_walk_flyweight():
    return calculate_std(RandomWalkIterable(10_000_000))

In [ ]:
memory_flyweight_1 = memory_usage(calculate_random_walk_flyweight)
memory_flyweight_2 = memory_usage(calculate_random_walk_flyweight)

In [ ]:
time_flyweight_1 = 0.1 * np.arange(len(memory_flyweight_1))
time_flyweight_2 = 0.1 * np.arange(len(memory_flyweight_2))
plt.plot(time_flyweight_1, memory_flyweight_1, label="flyweight 1")
plt.plot(time_flyweight_2, memory_flyweight_2, label="flyweight 2")
plt.xlabel("Time [s]")
plt.ylabel("Memory used [MB]")
plt.legend()
plt.savefig("figures/memory_used_flyweight.pdf")